In [49]:
# import all necessary libraries
import numpy as np
import cv2
import glob
import math
from datetime import datetime

# ---------------------------------------------------------------------------------------------------- CONFIGURATION

# function used to rotate a point around an origin point and a certain angle
def rotate ( origin , point , angle , height ) :
	angle = np.pi / 12 * (24 - angle)
	x = origin [ 0 ] + np.cos ( angle ) * (point [ 0 ] - origin [ 0 ]) - np.sin ( angle ) * (point [ 1 ] - origin [ 1 ])
	y = origin [ 1 ] + np.sin ( angle ) * (point [ 0 ] - origin [ 0 ]) + np.cos ( angle ) * (point [ 1 ] - origin [ 1 ])
	return [ round ( x ) , round ( y ) , round ( height ) ]

# variable to store which video we must elaborate, we must insert only numbers 1,2,3 or 4
videoToElaborate = 1

# load the matrix camera and distortion matrix after the calibration process
dist = np.load ( "framesVideoCalibration/uncalibrated/dist.npy" )
K = np.load ( "framesVideoCalibration/uncalibrated/K.npy" )

# get list of frames of video from the uncalibrated folder
listImages = glob.glob ( "framesVideo" + str ( videoToElaborate ) + "/uncalibrated/*.jpg" )

# list of all point coordinates of black polygon in order a,b,c,d,e
polygonCoordinates = np.array ( [ [ 70 , 0 , 0 ] , [ 65 , 5 , 0 ] , [ 98 , 5 , 0 ] , [ 98 , -5 , 0 ] , [ 65 , -5 , 0 ] ] ).squeeze ( ).astype ( "float32" )

# list of center coordinates of circles inside the black polygon
circlesCenterCoordinates = np.array ( [ [ 75 , 0 , 0 ] , [ 79.5 , 0 , 0 ] , [ 84 , 0 , 0 ] , [ 88.5 , 0 , 0 ] , [ 93 , 0 , 0 ] ] ).squeeze ( ).astype ( "float32" )

# 3d points list of all black polygons from the marker
all3dPoints = np.empty ( [ 24 , 5 , 3 ] )

# create a list of all points of the cube to project
listAllPointsOfCube = np.empty ( [ 90 * 90 * 60 , 3 ] )

# for each dimension of the cube
for i in range ( 0 , 90 ) :
	for j in range ( 0 , 90 ) :
		for k in range ( 0 , 60 ) :

			# append the actual point to the list
			listAllPointsOfCube [ i * 90 * 60 + j * 60 + k ] = np.array ( [ (i - 45) * 2 , (j - 45) * 2 , 70 + k * 2 ] )

# simplify the structure of the list of all points of the cube
listAllPointsOfCube = listAllPointsOfCube.squeeze ( )

# index 0 for the top structure of the list
index0 = 0

# for each black polygon in the marker
for i in range ( 0 , 24 ) :

	# list of coordinates of actual polygon
	all3dPointsActualPolygon = np.empty ( [ 5 , 3 ] )

	# index 1 for the inner structure of the list
	index1 = 0

	# for each coordinate of actual black polygon
	for polygonCoordinate in polygonCoordinates :

		# append the rotated coordinates
		all3dPointsActualPolygon [ index1 ] = np.array ( rotate ( (0 , 0 , 0) , (polygonCoordinate [ 0 ] , polygonCoordinate [ 1 ] , polygonCoordinate [ 2 ]) , i , 0 ) )

		# increment the index for the subarray
		index1 = index1 + 1

	# append the actual 3d coordinates in the final array
	all3dPoints [ index0 ] = all3dPointsActualPolygon

	# increment the index for the whole array
	index0 = index0 + 1

# create the cube as a linearized array
cube = np.zeros ( 90 * 90 * 60 )

# create the cube as a linearized array for each color
cubecolorsr = np.zeros ( 90 * 90 * 60 )
cubecolorsg = np.zeros ( 90 * 90 * 60 )
cubecolorsb = np.zeros ( 90 * 90 * 60 )

# for each image
for image in listImages :
	
	# open the actual image in color mode for the polygon borders algorithm
	imagePolygonBorders = cv2.imread ( image , cv2.IMREAD_COLOR )
	imagePolygonBorders = cv2.undistort ( imagePolygonBorders , K , dist )
	
	# open the actual image in color mode for the projection algorithm
	imageProjection = cv2.imread ( image , cv2.IMREAD_COLOR )
	imageProjection = cv2.undistort ( imageProjection , K , dist )
	
	# open the actual image in the background version
	imageBackgroundRemoved = cv2.imread ( image.replace ( "uncalibrated" , "backgroundremoved" ) , cv2.IMREAD_COLOR )
	imageBackgroundRemoved = cv2.undistort ( imageBackgroundRemoved , K , dist )
	
	# copy of the actual image to use it in the 3d model projection
	imageCube3d = cv2.imread ( image , cv2.IMREAD_COLOR )
	imageCube3d = cv2.undistort ( imageCube3d , K , dist )
	
	# open the actual image in gray mode
	gray_imagePolygonBorders = cv2.imread ( image , cv2.IMREAD_GRAYSCALE )
	gray_imagePolygonBorders = cv2.undistort ( gray_imagePolygonBorders , K , dist )
	
	# make binary image of gray image
	# If pixel intensity is greater than the set threshold, value set to 255 (white), else set to 0 (black)
	_ , threshold_gray_imagePolygonBorders = cv2.threshold ( gray_imagePolygonBorders , 190 , 255 , cv2.THRESH_BINARY )
	
	# save the threshold gray actual image in the binarythreshold folder
	cv2.imwrite ( image.replace ( "uncalibrated" , "binarythreshold" ) , threshold_gray_imagePolygonBorders )
	
	# get height and width to use them then with the new camera matrix calculation
	h , w = imagePolygonBorders.shape [ :2 ]
	newcameramtx , roi = cv2.getOptimalNewCameraMatrix ( K , dist , (w , h) , 1 , (w , h) )
	
	# ---------------------------------------------------------------------------------------------------- FINISH CONFIGURATION
	
	# ---------------------------------------------------------------------------------------------------- BLACK POLYGON AND WHITE CIRCLES DETECTION
	
	# detect shape in threshold gray actual image using regions with same colors and intensity
	# RETR_TREE to get a hierarchy of contours
	# CHAIN_APPROX_NONE all the boundary points are stored
	contours , hierarchy = cv2.findContours ( image = threshold_gray_imagePolygonBorders , mode = cv2.RETR_TREE , method = cv2.CHAIN_APPROX_NONE )
	
	# lists for 2d and 3d points
	list2dForSolvePnpFunction = [ ]
	list3dForSolvePnpFunction = [ ]
	
	# for each contour in threshold_gray_imagePolygonBorders
	for contour in contours :
		
		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )
		
		# approximate the shape of the actual polygon
		approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )
		
		# if the area of the actual region is bigger than a minimum value
		if area_actual_region > 3000 :
			
			# if we have 5 edges in the actual region
			if len ( approx ) == 5 :
				
				# calculate the perimeter of the contour
				perimeter = cv2.arcLength ( approx , True )
				
				# calculate the average of one edge
				avg_perimeter = perimeter / len ( approx )
				
				# save index of concave vertex
				first_edge_concave_vertex_clockwise = 0
				
				# for each edge
				for i in range ( 0 , len ( approx ) ) :
					
					# calculate the distance between the previous and the next one
					previous = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
					next = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
					
					# if they are both less than average edge
					if previous < avg_perimeter and next < avg_perimeter :
						
						# this is the concave vertex
						first_edge_concave_vertex_clockwise = i
						
						# exit from the for loop
						break
				
				# list of the coordinates of the points of the black polygon
				coordinatesActualPolygon = np.empty ( [ len ( approx ) , 2 ] )
				
				# for each edge
				for i in range ( 0 , len ( approx ) ) :
					
					# append in the list of the coordinates of the actual polygon the tuple (x,y) in order a,b,c,d,e
					coordinatesActualPolygon [ i ] = np.array ( [ approx [ (first_edge_concave_vertex_clockwise + i) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (first_edge_concave_vertex_clockwise + i) % len ( approx ) ] [ 0 ] [ 1 ] ] )
				
				# simplify the structure of the list
				coordinatesActualPolygon = coordinatesActualPolygon.squeeze ( )
				
				# solvepnp function with the references of actual polygon
				success , rotation_vector , translation_vector = cv2.solvePnP ( polygonCoordinates , coordinatesActualPolygon , newcameramtx , dist , cv2.SOLVEPNP_IPPE )
				
				# we can draw red contours of the actual region but in the colored image
				cv2.drawContours ( imagePolygonBorders , [ contour ] , 0 , (0 , 0 , 255) , 6 )
				
				# do the projection of the center of all circles of actual polygon
				projectedPoints , jacobian = cv2.projectPoints ( circlesCenterCoordinates , rotation_vector , translation_vector , newcameramtx , dist )
				# simplify the structure of given projected points
				projectedPoints = projectedPoints.squeeze ( )
				
				# array to store the binary value of actual polygon
				binaryValueActualPolygon = np.empty ( 5 )

				# index for binary values array
				binaryIndex = 0
				
				# for each circle center
				for circleCenter in projectedPoints :
					
					# if the actual center color is white
					if threshold_gray_imagePolygonBorders [ round ( circleCenter [ 1 ] ) ] [ round ( circleCenter [ 0 ] ) ] == 255 :
						
						# append a 0
						binaryValueActualPolygon [ binaryIndex ] = 0
					
					# else if the actual center color is black
					else :
						
						# append a 1
						binaryValueActualPolygon [ binaryIndex ] = 1
						
						# draw the center because is a valid circle for the binary value
						cv2.circle ( imagePolygonBorders , (round ( circleCenter [ 0 ] ) , round ( circleCenter [ 1 ] )) , 3 , (40 , 40 , 40) , -1 )

					# increment the index of binary array
					binaryIndex = binaryIndex + 1
				
				# convert the binary numpy array in integer value using sum of product between 0 and 1 values with the powers of 2
				actualIntegerValue = binaryValueActualPolygon.dot ( 2 ** np.arange ( binaryValueActualPolygon.size ) )
				
				# calculate the edge of the outer part of the polygon
				second = approx [ (first_edge_concave_vertex_clockwise + 2) % len ( approx ) ] [ 0 ]
				third = approx [ (first_edge_concave_vertex_clockwise + 3) % len ( approx ) ] [ 0 ]
				
				# calculate the middle point of the previous edge
				half_second_third = [ (second [ 0 ] + third [ 0 ]) // 2 , (second [ 1 ] + third [ 1 ]) // 2 ]
				
				# insert the integer value of the actual polygon
				cv2.putText ( imagePolygonBorders , str ( int ( actualIntegerValue ) ) , (round ( half_second_third [ 0 ] ) , round ( half_second_third [ 1 ] )) , cv2.FONT_ITALIC , 1 , (0 , 0 , 0) , 6 , cv2.LINE_4 , False )
				
				# for each 2d coordinates of the actual polygon
				for coordinate2dActualPolygon in coordinatesActualPolygon :
					
					# i insert them in the final array for the final projection
					list2dForSolvePnpFunction.append ( [ coordinate2dActualPolygon [ 0 ] , coordinate2dActualPolygon [ 1 ] ] )
				
				# for each 3d coordinates of the actual polygon
				for coordinate3dActualPolygon in all3dPoints [ int ( actualIntegerValue ) ] :
					
					# i insert them in the final array for the final projection
					list3dForSolvePnpFunction.append ( [ coordinate3dActualPolygon [ 0 ] , coordinate3dActualPolygon [ 1 ] , coordinate3dActualPolygon [ 2 ] ] )
	
	# save the image with the detected polygon
	cv2.imwrite ( image.replace ( "uncalibrated" , "polygonborders" ) , imagePolygonBorders )

	# ---------------------------------------------------------------------------------------------------- FINISH BLACK POLYGON AND WHITE CIRCLES DETECTION
	
	# ---------------------------------------------------------------------------------------------------- PROJECTION BLACK POLYGON WITH THEIR VALUES

	# make standard the two array for the solvepnp function
	list2dForSolvePnpFunction = np.array ( list2dForSolvePnpFunction ).squeeze ( ).astype ( "float32" )
	list3dForSolvePnpFunction = np.array ( list3dForSolvePnpFunction ).squeeze ( ).astype ( "float32" )
	
	# associate the 2d and 3d array in the solvepnp function
	success , rotation_vector , translation_vector = cv2.solvePnP ( list3dForSolvePnpFunction , list2dForSolvePnpFunction , newcameramtx , dist , cv2.SOLVEPNP_IPPE )
	
	# project all the black polygon in a new image
	projectedPoints , jacobian = cv2.projectPoints ( all3dPoints.reshape ( 120 , 3 ).squeeze ( ).astype ( "float32" ) , rotation_vector , translation_vector , newcameramtx , dist )
	
	# simplify the structure of given projected points
	projectedPoints = projectedPoints.reshape ( 24 , 5 , 2 ).squeeze ( ).astype ( "int" )
	
	# store the integer value of the actual polygon
	actualIntegerValue = 0
	
	# for each projected black polygon
	for projectedPoint in projectedPoints :
		
		# for each edge
		for i in range ( 0 , 5 ) :
			
			# draw the edge
			cv2.line ( imageProjection , (projectedPoint [ i ] [ 0 ] - 0 , projectedPoint [ i ] [ 1 ]) , (projectedPoint [ (i + 1) % 5 ] [ 0 ] - 0 , projectedPoint [ (i + 1) % 5 ] [ 1 ]) , (0 , 0 , 255) , thickness = 6 )
		
		# calculate the edge of the outer part of the polygon
		second = projectedPoint [ 2 ]
		third = projectedPoint [ 3 ]
		
		# calculate the middle point of the previous edge
		half_second_third = [ (second [ 0 ] + third [ 0 ]) // 2 , (second [ 1 ] + third [ 1 ]) // 2 ]
		
		# insert the integer value of the actual polygon
		cv2.putText ( imageProjection , str ( actualIntegerValue % 24 ) , (round ( half_second_third [ 0 ] ) , round ( half_second_third [ 1 ] )) , cv2.FONT_ITALIC , 1 , (0 , 0 , 0) , 6 , cv2.LINE_4 , False )
		
		# increment the integer value for the next polygon
		actualIntegerValue = actualIntegerValue + 1
	
	# save the image with detected borders
	cv2.imwrite ( image.replace ( "uncalibrated" , "projectionpoints" ) , imageProjection )

	# ---------------------------------------------------------------------------------------------------- FINISH PROJECTION BLACK POLYGON WITH THEIR VALUES
	
	# ---------------------------------------------------------------------------------------------------- PROJECTION POINTS IN OR OUT THE OBJECT

	# project all points
	projectedPoints , jacobian = cv2.projectPoints ( listAllPointsOfCube , rotation_vector , translation_vector , newcameramtx , dist )

	#simplify the structure of the projected points
	projectedPoints = projectedPoints.squeeze ( )
	
	# index for the actual projected point
	index = 0

	# for each projected point
	for projectedPoint in projectedPoints :

		# round the coordinates of the actual point and create a valid point coordinates
		projectedPoint = [ round ( projectedPoint [ 0 ] ) , round ( projectedPoint [ 1 ] ) ]
		
		# if the coordinates are into the dimensions of the image 1080x1920
		if 0 <= projectedPoint [ 1 ] < 1080 and 0 <= projectedPoint [ 0 ] < 1920 :
			
			# if the actual coordinates cube value is not checked or inside the object
			if cube [ index ] == 0 or cube [ index ] == 1 :
				
				# check if the actual position is 000 that in rgb value means black because we set from background deletion algorithm the object as a black obj
				if np.array_equal ( imageBackgroundRemoved [ projectedPoint [ 1 ] ] [ projectedPoint [ 0 ] ] , [ 0 , 0 , 0 ] ) :
					
					# if the actual pixel is black means that is into the object
					cube [ index ] = 1
					
					# if not checked before the actual point
					if cubecolorsr [ index ] == 0 and cubecolorsg [ index ] == 0 and cubecolorsb [ index ] == 0 :
						
						# save the color of actual pixel
						cubecolorsr [ index ] = imageCube3d [ projectedPoint [ 1 ] , projectedPoint [ 0 ] ] [ 0 ]
						cubecolorsg [ index ] = imageCube3d [ projectedPoint [ 1 ] , projectedPoint [ 0 ] ] [ 1 ]
						cubecolorsb [ index ] = imageCube3d [ projectedPoint [ 1 ] , projectedPoint [ 0 ] ] [ 2 ]
				
				# the actual pixel is not black
				else :
					
					# remove the actual pixel from the cube
					cube [ index ] = 2
					
					# save the color of actual pixel
					cubecolorsr [ index ] = 0
					cubecolorsg [ index ] = 0
					cubecolorsb [ index ] = 0
		
		# increment the index for the next projected points
		index = index + 1

# ---------------------------------------------------------------------------------------------------- FINISH PROJECTION POINTS IN OR OUT THE OBJECT

# ---------------------------------------------------------------------------------------------------- CREATION PLY FILE

# variable used to store the number of vertices
contVertices = 0

# variable used to store the number of faces
contFaces = 0

# for each dimension of the cube
for i in range ( 0 , 90 ) :
	for j in range ( 0 , 90 ) :
		for k in range ( 0 , 60 ) :

			# if the actual position is inside the object
			if cube [ i * 90 * 60 + j * 60 + k ] == 1 :

				# add 8 vertices in the counter because a cube has 8 vertices
				contVertices = contVertices + 8

				# add 6 faces because a cube has 6 faces
				contFaces = contFaces + 6

# store the header of the ply file
plyFileHeader = np.empty ( 12 ).astype ( "object" )

# store vertices of the ply file
plyFileVertices = np.empty ( contVertices ).astype ( "object" )

# store faces of the ply file
plyFileCube = np.empty ( contFaces ).astype ( "object" )

# write into the ply file the whole header
plyFileHeader [ 0 ] = "ply"
plyFileHeader [ 1 ] = "format ascii 1.0"
plyFileHeader [ 2 ] = "element vertex " + str ( contVertices )
plyFileHeader [ 3 ] = "property float x"
plyFileHeader [ 4 ] = "property float y"
plyFileHeader [ 5 ] = "property float z"
plyFileHeader [ 6 ] = "element face " + str ( contFaces )
plyFileHeader [ 7 ] = "property list uint8 int vertex_indices"
plyFileHeader [ 8 ] = "property uchar red"
plyFileHeader [ 9 ] = "property uchar green"
plyFileHeader [ 10 ] = "property uchar blue"
plyFileHeader [ 11 ] = "end_header"

# variable used to store the sum of vertices of cube
sumVerticesCube = 0

# variable used to count the actual pixel block on the cube where there is the object
index = 0

# for each dimension of the cube
for i in range ( 0 , 90 ) :
	for j in range ( 0 , 90 ) :
		for k in range ( 0 , 60 ) :

			# if the actual position is inside the object
			if cube [ i * 90 * 60 + j * 60 + k ] == 1 :

				# list of all vertices
				listAllVertices = np.array ( [ str ( 0 + i ) + " " + str ( 0 + j ) + " " + str ( 0 + k ) ,
											   str ( 2 + i ) + " " + str ( 0 + j ) + " " + str ( 0 + k ) ,
											   str ( 2 + i ) + " " + str ( 2 + j ) + " " + str ( 0 + k ) ,
											   str ( 0 + i ) + " " + str ( 2 + j ) + " " + str ( 0 + k ) ,
											   str ( 0 + i ) + " " + str ( 0 + j ) + " " + str ( 1 + k ) ,
											   str ( 2 + i ) + " " + str ( 0 + j ) + " " + str ( 1 + k ) ,
											   str ( 2 + i ) + " " + str ( 2 + j ) + " " + str ( 1 + k ) ,
											   str ( 0 + i ) + " " + str ( 2 + j ) + " " + str ( 1 + k ) ] )

				# list of all faces
				listAllFaces = np.array ( [ "4 " + str ( sumVerticesCube + 0 ) + " " + str ( sumVerticesCube + 1 ) + " " + str ( sumVerticesCube + 2 ) + " " + str ( sumVerticesCube + 3 )
											+ " " + str ( round ( cubecolorsb [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsg [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsr [ i * 90 * 60 + j * 60 + k ] ) ) ,
											"4 " + str ( sumVerticesCube + 4 ) + " " + str ( sumVerticesCube + 5 ) + " " + str ( sumVerticesCube + 6 ) + " " + str ( sumVerticesCube + 7 )
											+ " " + str ( round ( cubecolorsb [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsg [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsr [ i * 90 * 60 + j * 60 + k ] ) ) ,
											"4 " + str ( sumVerticesCube + 1 ) + " " + str ( sumVerticesCube + 2 ) + " " + str ( sumVerticesCube + 6 ) + " " + str ( sumVerticesCube + 5 ) + " " + str ( round ( cubecolorsb [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsg [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsr [ i * 90 * 60 + j * 60 + k ] ) ) ,
											"4 " + str ( sumVerticesCube + 2 ) + " " + str ( sumVerticesCube + 3 ) + " " + str ( sumVerticesCube + 7 ) + " " + str ( sumVerticesCube + 6 )
											+ " " + str ( round ( cubecolorsb [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsg [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsr [ i * 90 * 60 + j * 60 + k ] ) ) ,
											"4 " + str ( sumVerticesCube + 3 ) + " " + str ( sumVerticesCube + 0 ) + " " + str ( sumVerticesCube + 4 ) + " " + str ( sumVerticesCube + 7 )
											+ " " + str ( round ( cubecolorsb [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsg [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsr [ i * 90 * 60 + j * 60 + k ] ) ) ,
											"4 " + str ( sumVerticesCube + 0 ) + " " + str ( sumVerticesCube + 1 ) + " " + str ( sumVerticesCube + 5 ) + " " + str ( sumVerticesCube + 4 )
											+ " " + str ( round ( cubecolorsb [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsg [ i * 90 * 60 + j * 60 + k ] ) ) + " " + str ( round ( cubecolorsr [ i * 90 * 60 + j * 60 + k ] ) ) ] )

				# for each of 8 vertices of each cube
				for l in range ( 0 , 8 ) :

					# add the current vertex to the relative list
					plyFileVertices [ index * 8 + l ] = listAllVertices [ l ]

				# for each of 6 faces of each cube
				for l in range ( 0 , 6 ) :

					# add the current vertex to the relative list
					plyFileCube [ index * 6 + l ] = listAllFaces [ l ]

				# shift the faces of the actual cube using the 8 vertices for each cube
				sumVerticesCube = sumVerticesCube + 8

				index = index + 1

# open the ply file
with open ( "framesVideo" + str ( videoToElaborate ) + "/output/file.ply" , "a" ) as f :

	# save in the file all parts of the ply file
	np.savetxt ( f , plyFileHeader , delimiter = " " , fmt = "%s" )
	np.savetxt ( f , plyFileVertices , delimiter = " " , fmt = "%s" )
	np.savetxt ( f , plyFileCube , delimiter = " " , fmt = "%s" )

# ---------------------------------------------------------------------------------------------------- FINISH CREATION PLY FILE
